In [8]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras import Sequential
import numpy as np

## Data Creation 

In [9]:
import pandas as pd
import numpy as np
# Set a seed for reproducibility
np.random.seed(42)
# Number of samples
num_samples = 20000
# Create a DataFrame with 6 random features
df = pd.DataFrame({
    'feature1': np.random.rand(num_samples),
    'feature2': np.random.rand(num_samples),
    'feature3': np.random.rand(num_samples),
    'feature4': np.random.rand(num_samples),
    'feature5': np.random.rand(num_samples),
    'feature6': np.random.rand(num_samples),
    'output': np.random.choice([0, 1, 2], size=num_samples)  # Randomly assign one of three classes
})
# Display the first few rows of the DataFrame
df.head()

,feature1,feature2,feature3,feature4,feature5,feature6,output
0,0.374540,0.729998,0.298912,0.741555,0.818164,0.580779,1
1,0.950714,0.184512,0.094818,0.881102,0.145270,0.526972,0
2,0.731994,0.346640,0.126359,0.463180,0.946464,0.351037,1
3,0.598658,0.663281,0.180671,0.289179,0.843224,0.493213,0
4,0.156019,0.482089,0.203653,0.318847,0.918853,0.365097,2


## Part-1 (LSTM model without return_sequences=False)

### Data Preparation

In [ ]:


from sklearn.model_selection import train_test_split
# Step 1: Create Sequences and target 
sequence_length = 5 # time lags or time steps 
sequences = []
targets= []
for i in range(len(df) - sequence_length-1):
    sequences.append(df.iloc[i:i+sequence_length].values)
    targets.append(df.iloc[i+sequence_length+1].values[-1])
#print(np.shape(sequences)) #(samples, time steps,features) here features consitute have features + output
#print(np.shape(targets)) # (samples,)

# Step-2 Splitting the data into test_train_validation set 
X_train, X_test, y_train, y_test = train_test_split(sequences, targets, test_size=0.2, random_state=42, stratify=targets)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)
# Step 4: Reshape Data
X_train = np.array(X_train)
X_test = np.array(X_test)
X_val = np.array(X_val)
y_train = np.array(y_train)
y_test = np.array(y_test)
y_val = np.array(y_val)

# Reshape X_train, X_test,X_val to 3D arrays (samples, time steps, features)
X_train = X_train.reshape(X_train.shape[0], sequence_length, X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], sequence_length, X_test.shape[2])
X_val = X_val.reshape(X_val.shape[0], sequence_length, X_val.shape[2])

print("Shape of X_train data set",X_train.shape) 
print("Shape of y_train data set",y_train.shape)

### Model 

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
# Define LSTM model
model = Sequential()
model.add(LSTM(units=4,return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(units=3, activation='softmax'))

# Compile the model with class weights
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
### Model Training 

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler

# Define early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='min', 
                               restore_best_weights=True)

# Define dynamic learning rate scheduler
def lr_schedule(epoch):
    lr = 0.1
    if epoch > 50:
        lr *= 0.5
    elif epoch > 75:
        lr *= 0.1
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)

# Compute class weights ( necessary if you have unbalanced datasets)
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {class_index: weight for class_index, weight in enumerate(class_weights)}

# Fit the model 
history = model.fit(X_train, y_train, epochs=20, batch_size=256, validation_data=(X_test, y_test), 
                    callbacks=[early_stopping, lr_scheduler],class_weight=class_weights_dict,verbose=1)


### Visualization 


In [ ]:
import matplotlib.pyplot as plt
# Extract loss history and learning rates
train_loss = history.history['loss']
val_loss = history.history['val_loss']
learning_rates = [lr_schedule(epoch) for epoch in range(1, len(train_loss) + 1)]
epochs = range(1, len(train_loss) + 1)

# Plot loss with respect to epochs
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs, train_loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot learning rate with respect to epochs
plt.subplot(1, 2, 2)
plt.plot(epochs, learning_rates, 'g', label='Learning Rate')
plt.title('Learning Rate')
plt.xlabel('Epochs')
plt.ylabel('Learning Rate')
plt.legend()

plt.tight_layout()
plt.show()

### Prediction on test set 

In [ ]:
y_predict = model.predict(X_test)
y_predict = np.argmax(y_predict)


## Part-2 (LSTM model without return_sequences=True)

### Data Preparation

In [ ]:
from sklearn.model_selection import train_test_split
# Step 1: Create Sequences and target 
sequence_length = 5 # time lags or time steps 
sequences = []
targets= []
for i in range(len(df) - sequence_length-1):
    sequences.append(df.iloc[i:i+sequence_length].values)
    targets.append(df.iloc[i+1:i+1+sequence_length,-1]) # only change happened here 
#print(np.shape(sequences)) #(samples, time steps,features) here features consitute have features + output
#print(np.shape(targets)) # (samples,)

# Step-2 Splitting the data into test_train_validation set 
X_train, X_test, y_train, y_test = train_test_split(sequences, targets, test_size=0.2, random_state=42, stratify=targets)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)
# Step 4: Reshape Data
X_train = np.array(X_train)
X_test = np.array(X_test)
X_val = np.array(X_val)
y_train = np.array(y_train)
y_test = np.array(y_test)
y_val = np.array(y_val)

# Reshape X_train, X_test,X_val to 3D arrays (samples, time steps, features)
X_train = X_train.reshape(X_train.shape[0], sequence_length, X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], sequence_length, X_test.shape[2])
X_val = X_val.reshape(X_val.shape[0], sequence_length, X_val.shape[2])

print("Shape of X_train data set",X_train.shape) 
print("Shape of y_train data set",y_train.shape)

### modelling 

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
# Define LSTM model
model = Sequential()
model.add(LSTM(units=4,return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(units=3, activation='softmax'))
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

### Training 

In [ ]:


from sklearn.utils.class_weight import compute_class_weight
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
import matplotlib.pyplot as plt





# Define early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='min', 
                               restore_best_weights=True)

# Define dynamic learning rate scheduler
def lr_schedule(epoch):
    lr = 0.1
    if epoch > 50:
        lr *= 0.5
    elif epoch > 75:
        lr *= 0.1
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)


# Fit the model 
history = model.fit(X_train, y_train, epochs=10, batch_size=256, validation_data=(X_test, y_test), 
                    callbacks=[early_stopping, lr_scheduler])



### Visualizing 

In [ ]:
# Extract loss history and learning rates
train_loss = history.history['loss']
val_loss = history.history['val_loss']
learning_rates = [lr_schedule(epoch) for epoch in range(1, len(train_loss) + 1)]
epochs = range(1, len(train_loss) + 1)

# Plot loss with respect to epochs
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs, train_loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot learning rate with respect to epochs
plt.subplot(1, 2, 2)
plt.plot(epochs, learning_rates, 'g', label='Learning Rate')
plt.title('Learning Rate')
plt.xlabel('Epochs')
plt.ylabel('Learning Rate')
plt.legend()

plt.tight_layout()
plt.show()